# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [582]:
import pandas as pd

In [583]:
data = pd.read_csv('../data/cars.csv')
data

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,True,False,True,True,False,False,False,False,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,Chrysler,300,automatic,silver,290000,2000,gasoline,False,gasoline,3.5,...,True,False,False,True,True,False,False,True,True,301
38527,Chrysler,PT Cruiser,mechanical,blue,321000,2004,diesel,False,diesel,2.2,...,True,False,False,True,True,False,False,True,True,317
38528,Chrysler,300,automatic,blue,777957,2000,gasoline,False,gasoline,3.5,...,True,False,False,True,True,False,False,True,True,369
38529,Chrysler,PT Cruiser,mechanical,black,20000,2001,gasoline,False,gasoline,2.0,...,True,False,False,False,False,False,False,False,True,490


In [584]:
data.isnull().any()

manufacturer_name    False
model_name           False
transmission         False
color                False
odometer_value       False
year_produced        False
engine_fuel          False
engine_has_gas       False
engine_type          False
engine_capacity       True
body_type            False
has_warranty         False
state                False
drivetrain           False
price_usd            False
is_exchangeable      False
location_region      False
number_of_photos     False
up_counter           False
feature_0            False
feature_1            False
feature_2            False
feature_3            False
feature_4            False
feature_5            False
feature_6            False
feature_7            False
feature_8            False
feature_9            False
duration_listed      False
dtype: bool

In [585]:
data['engine_capacity'].fillna(data['engine_capacity'].mean(), inplace=True)

In [586]:
data.isnull().any()

manufacturer_name    False
model_name           False
transmission         False
color                False
odometer_value       False
year_produced        False
engine_fuel          False
engine_has_gas       False
engine_type          False
engine_capacity      False
body_type            False
has_warranty         False
state                False
drivetrain           False
price_usd            False
is_exchangeable      False
location_region      False
number_of_photos     False
up_counter           False
feature_0            False
feature_1            False
feature_2            False
feature_3            False
feature_4            False
feature_5            False
feature_6            False
feature_7            False
feature_8            False
feature_9            False
duration_listed      False
dtype: bool

In [587]:
data.drop_duplicates(inplace=True)

In [588]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38491 entries, 0 to 38530
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manufacturer_name  38491 non-null  object 
 1   model_name         38491 non-null  object 
 2   transmission       38491 non-null  object 
 3   color              38491 non-null  object 
 4   odometer_value     38491 non-null  int64  
 5   year_produced      38491 non-null  int64  
 6   engine_fuel        38491 non-null  object 
 7   engine_has_gas     38491 non-null  bool   
 8   engine_type        38491 non-null  object 
 9   engine_capacity    38491 non-null  float64
 10  body_type          38491 non-null  object 
 11  has_warranty       38491 non-null  bool   
 12  state              38491 non-null  object 
 13  drivetrain         38491 non-null  object 
 14  price_usd          38491 non-null  float64
 15  is_exchangeable    38491 non-null  bool   
 16  location_region    384

In [589]:
dum_data = pd.get_dummies(data, columns=['transmission', 'engine_type', 'has_warranty', 'is_exchangeable', 'engine_has_gas', 'body_type', 'state', 'engine_fuel', 'drivetrain', 'location_region'], drop_first=True)
dum_data

,manufacturer_name,model_name,color,odometer_value,year_produced,engine_capacity,price_usd,number_of_photos,up_counter,feature_0,...,engine_fuel_gasoline,engine_fuel_hybrid-diesel,engine_fuel_hybrid-petrol,drivetrain_front,drivetrain_rear,location_region_Витебская обл.,location_region_Гомельская обл.,location_region_Гродненская обл.,location_region_Минская обл.,location_region_Могилевская обл.
0,Subaru,Outback,silver,190000,2010,2.5,10900.00,9,13,False,...,1,0,0,0,0,0,0,0,1,0
1,Subaru,Outback,blue,290000,2002,3.0,5000.00,12,54,False,...,1,0,0,0,0,0,0,0,1,0
2,Subaru,Forester,red,402000,2001,2.5,2800.00,4,72,False,...,1,0,0,0,0,0,0,0,1,0
3,Subaru,Impreza,blue,10000,1999,3.0,9999.00,9,42,True,...,1,0,0,0,0,0,0,0,1,0
4,Subaru,Legacy,black,280000,2001,2.5,2134.11,14,7,False,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,Chrysler,300,silver,290000,2000,3.5,2750.00,5,85,False,...,1,0,0,1,0,0,0,0,1,0
38527,Chrysler,PT Cruiser,blue,321000,2004,2.2,4800.00,4,20,False,...,0,0,0,1,0,0,0,0,0,0
38528,Chrysler,300,blue,777957,2000,3.5,4300.00,3,63,False,...,1,0,0,1,0,0,0,0,1,0
38529,Chrysler,PT Cruiser,black,20000,2001,2.0,4000.00,7,156,False,...,1,0,0,1,0,0,0,0,0,0


In [590]:
dum_data.drop(['manufacturer_name', 'model_name', 'number_of_photos', 'color', 'up_counter'], axis=1, inplace=True)

In [591]:
dum_data.to_csv('../data/dum_data.csv')

In [592]:
y = dum_data['price_usd']
X = dum_data.drop(['price_usd'], axis=1)

In [593]:
y

0        10900.00
1         5000.00
2         2800.00
3         9999.00
4         2134.11
           ...   
38526     2750.00
38527     4800.00
38528     4300.00
38529     4000.00
38530     3200.00
Name: price_usd, Length: 38491, dtype: float64

In [594]:
X

,odometer_value,year_produced,engine_capacity,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,engine_fuel_gasoline,engine_fuel_hybrid-diesel,engine_fuel_hybrid-petrol,drivetrain_front,drivetrain_rear,location_region_Витебская обл.,location_region_Гомельская обл.,location_region_Гродненская обл.,location_region_Минская обл.,location_region_Могилевская обл.
0,190000,2010,2.5,False,True,True,True,False,True,False,...,1,0,0,0,0,0,0,0,1,0
1,290000,2002,3.0,False,True,False,False,True,True,False,...,1,0,0,0,0,0,0,0,1,0
2,402000,2001,2.5,False,True,False,False,False,False,False,...,1,0,0,0,0,0,0,0,1,0
3,10000,1999,3.0,True,False,False,False,False,False,False,...,1,0,0,0,0,0,0,0,1,0
4,280000,2001,2.5,False,True,False,True,True,False,False,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,3.5,False,True,False,False,True,True,False,...,1,0,0,1,0,0,0,0,1,0
38527,321000,2004,2.2,False,True,False,False,True,True,False,...,0,0,0,1,0,0,0,0,0,0
38528,777957,2000,3.5,False,True,False,False,True,True,False,...,1,0,0,1,0,0,0,0,1,0
38529,20000,2001,2.0,False,True,False,False,False,False,False,...,1,0,0,1,0,0,0,0,0,0


In [595]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [596]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((30792, 45), (30792,), (7699, 45), (7699,))

In [597]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
lr = LinearRegression().fit(X_train, y_train)

In [598]:
lr.predict(X_test)

array([15976.94084928, 23810.10408517, 10431.47429683, ...,
       11585.40399477,   760.69900116, 15526.95369111])

In [599]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

In [600]:
y_pred = lr.predict(X_test)

In [601]:
len(y_pred)

7699

In [602]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 2084.1179267180614
MSE: 10639972.212870665
RMSE: 3261.8970267117056
MAPE: 1.4305322578336521
R^2: 0.7247604344954139


In [603]:
len(lr.coef_)
lr.coef_

array([-5.06020523e-03,  3.68309736e+02,  8.76307967e+02,  2.84860544e+02,
       -8.42068163e+02,  5.53901252e+02,  7.63486276e+02,  7.33244204e+01,
        1.48563999e+02,  1.20006689e+03,  1.28234045e+03,  3.06860409e+02,
        1.44348895e+02,  1.35489326e+00, -7.71799757e+02,  8.57513388e+02,
       -3.70930079e+02,  1.90527187e+03, -1.63487450e+02, -5.18036310e+02,
       -1.90746068e+03, -3.54384792e+03, -2.72162122e+03, -3.35628570e+03,
       -7.48024202e+01, -2.73462456e+03, -4.32909677e+01, -3.13799186e+03,
       -1.42008666e+03, -3.47037347e+03, -1.70910330e+03,  1.19125903e+04,
        1.49782382e+03,  8.57513388e+02, -5.18036310e+02, -5.13872756e+02,
        2.05274551e+03,  6.60978987e+02, -2.43393644e+03, -1.24472131e+03,
       -3.16763321e+02, -1.39132524e+02, -2.70602194e+01,  3.27149275e+02,
       -1.38353454e+02])

In [604]:
ridge = Ridge(alpha=0.9).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 2084.2483203895035
MSE: 10642133.805227755
RMSE: 3262.2283496450327
MAPE: 1.4304597854416032
R^2: 0.7247045174564162


array([-5.06605442e-03,  3.68409024e+02,  8.75794613e+02,  2.86210360e+02,
       -8.41762922e+02,  5.54443877e+02,  7.62896419e+02,  7.51692492e+01,
        1.48251771e+02,  1.19970974e+03,  1.28113912e+03,  3.06328665e+02,
        1.44944398e+02,  1.36018977e+00, -7.72473133e+02,  8.10943224e+02,
       -3.72130623e+02,  1.96974369e+03, -1.62679216e+02, -5.17283594e+02,
       -1.59125209e+03, -3.23025723e+03, -2.40381116e+03, -2.79166709e+03,
        2.38801331e+02, -2.42005764e+03,  2.68903113e+02, -2.82446176e+03,
       -1.10553674e+03, -3.15608457e+03, -1.39410413e+03,  1.17818893e+04,
        1.45732166e+03,  8.10943224e+02, -5.17283594e+02, -5.12515982e+02,
        1.39965587e+03,  6.57668953e+02, -2.43238193e+03, -1.24148206e+03,
       -3.16526478e+02, -1.38392286e+02, -2.70076292e+01,  3.28433530e+02,
       -1.37351998e+02])

In [605]:
lasso = Lasso(alpha=0.9).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 2085.0896290566834
MSE: 10665477.24242328
RMSE: 3265.8042259791505
MAPE: 1.4268840318592377
R^2: 0.7241006589704599


array([-5.09330469e-03,  3.69242032e+02,  8.79326855e+02,  2.78095694e+02,
       -8.41592665e+02,  5.51569860e+02,  7.57979498e+02,  7.80921135e+01,
        1.45921174e+02,  1.19829114e+03,  1.28037614e+03,  3.00010447e+02,
        1.39182388e+02,  1.37683096e+00, -7.74438498e+02,  0.00000000e+00,
       -8.84311929e+02,  1.85113968e+03, -1.55632954e+02, -4.93224287e+02,
       -1.37943624e+02, -1.81906240e+03, -9.33308036e+02, -0.00000000e+00,
        1.61274647e+03, -1.00829064e+03,  1.38877556e+03, -1.41843735e+03,
        2.94472793e+02, -1.74534610e+03, -0.00000000e+00,  1.16678154e+04,
        1.31269213e+03,  0.00000000e+00, -5.26094156e+00, -1.81343369e+00,
        0.00000000e+00,  1.01779134e+03, -2.42642456e+03, -1.21734211e+03,
       -2.78863813e+02, -9.82416675e+01, -0.00000000e+00,  3.58540924e+02,
       -9.56644677e+01])

In [606]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np

In [607]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

In [608]:
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
# выводим оптимальные значения параметров
ridge_optimal.best_params_

{'alpha': 0.9}

In [609]:
ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train, y_train)
# выводим оптимальные значения параметров
ridge_optimal.best_params_

{'alpha': 0.9}

In [610]:
from sklearn.preprocessing import PolynomialFeatures
# создаем объект, который позволит расширить множество предикторов
p = PolynomialFeatures(2)  
# добавляем новые предикторы
X_p=p.fit_transform(X) 
X_p

array([[1.00000e+00, 1.90000e+05, 2.01000e+03, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 2.90000e+05, 2.00200e+03, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 4.02000e+05, 2.00100e+03, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [1.00000e+00, 7.77957e+05, 2.00000e+03, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 2.00000e+04, 2.00100e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 2.97729e+05, 2.00000e+03, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [611]:
X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(X_p, y, test_size=0.2)

In [612]:
X_ptrain.shape, y_ptrain.shape, X_ptest.shape, y_ptest.shape

((30792, 1081), (30792,), (7699, 1081), (7699,))

In [613]:
lr2 = LinearRegression().fit(X_ptrain, y_ptrain)
lr2.coef_

array([ 8.50175195e+04,  5.43575753e-01, -5.52461263e+04, ...,
        3.14053370e+04,  0.00000000e+00,  5.66793274e+04])

In [614]:
lr2.predict(X_ptest)

array([ 1566.9410726 , 17648.57632804,  6533.20395462, ...,
       20999.29987714,  3247.69799992,  2706.61282767])

In [615]:
y_ppred = lr2.predict(X_ptest)

In [616]:
len(y_ppred)

7699

In [617]:
print(f'MAE: {mean_absolute_error(y_ptest, y_ppred)}')
print(f'MSE: {mean_squared_error(y_ptest, y_ppred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_ptest, y_ppred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_ptest, y_ppred))}')
print(f'R^2: {lr2.score(X_ptest, y_ptest)}')


# print(f'MAE: {np.mean(abs(y-y_pred))}')
# print(f'MSE: {np.mean((y-y_pred)**2)}')
# print(f'RMSE: {np.sqrt(np.mean(np.square(y-y_pred)))}')
# print(f'MAPE: {np.mean(abs((y-y_pred)/y))}')
# print(f'R^2: {lr2.score(X_p, y)}')

MAE: 1540.1543364185245
MSE: 6705786.675667325
RMSE: 2589.553373782306
MAPE: 0.8437127441552482
R^2: 0.8367986076555405


In [626]:
pridge = Ridge(alpha=0.5).fit(X_ptrain, y_ptrain)
y_ppred = pridge.predict(X_ptest)
print(f'MAE: {mean_absolute_error(y_ptest, y_ppred)}')
print(f'MSE: {mean_squared_error(y_ptest, y_ppred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_ptest, y_ppred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_ptest, y_ppred))}')
print(f'R^2: {pridge.score(X_ptest, y_ptest)}')
pridge.coef_

MAE: 1577.6264877549754
MSE: 6702129.6915633455
RMSE: 2588.847174238631
MAPE: 0.8864687081394965
R^2: 0.8368876091294043


D:\jupyter\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.4506e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


array([ 0.00000000e+00,  3.82027060e-01, -5.95664924e+04, ...,
       -4.59545791e+03,  0.00000000e+00,  1.84916414e+03])

In [627]:
plasso = Lasso(alpha=0.5).fit(X_ptrain, y_ptrain)
y_ppred = plasso.predict(X_ptest)
print(f'MAE: {mean_absolute_error(y_ptest, y_ppred)}')
print(f'MSE: {mean_squared_error(y_ptest, y_ppred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_ptest, y_ppred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_ptest, y_ppred))}')
print(f'R^2: {plasso.score(X_ptest, y_ptest)}')
plasso.coef_

MAE: 1803.2553281372616
MSE: 8322832.476187416
RMSE: 2884.9319708075295
MAPE: 1.0671425195704096
R^2: 0.79744392208416


D:\jupyter\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.242e+11, tolerance: 1.272e+08
  model = cd_fast.enet_coordinate_descent(


array([ 0.00000000e+00, -1.11719272e-02, -3.21106317e+01, ...,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00])